In [179]:
import requests
import pandas as pd
import janitor
import itertools

In [180]:
# Kommunevalg 2019, 9 største partier, i alle 2020 kommuner 
dataset = 10804
query_url = "https://www.ssb.no/statbank/sq/10082984" # generert på ssb.no

data = requests.get(query_url).json()

In [181]:
def grouper(n, iterable, fillvalue=None):
    # https://stackoverflow.com/a/2231713/5422525
    """grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"""
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args)

In [197]:
import numpy as np

In [183]:
party_names

dict_values(['Arbeiderpartiet', 'Fremskrittspartiet', 'Høyre', 'Kristelig Folkeparti', 'Miljøpartiet de Grønne', 'Rødt', 'Senterpartiet', 'Sosialistisk Venstreparti', 'Venstre'])

In [241]:
y

NameError: name 'y' is not defined

,Arbeiderpartiet,Fremskrittspartiet,Høyre,Kristelig Folkeparti,Miljøpartiet de Grønne,Rødt,Senterpartiet,Sosialistisk Venstreparti,Venstre
0,0.192359,0.076411,0.306015,0.035324,0.078185,0.077150,0.150532,0.054981,0.029042
1,0.312092,0.098618,0.214563,0.027223,0.066376,0.079585,0.081432,0.067749,0.052362
2,0.424196,0.127499,0.120590,0.049196,0.048804,0.051842,0.132840,0.029743,0.015288
3,0.394290,0.136044,0.165726,0.038340,0.075261,0.054209,0.072134,0.048099,0.015898
4,0.289306,0.118372,0.283716,0.024843,0.095036,0.027813,0.088123,0.049687,0.023104
...,...,...,...,...,...,...,...,...,...
351,0.480591,0.000000,0.188540,0.000000,0.000000,0.000000,0.109057,0.221811,0.000000
352,0.450034,0.083617,0.073419,0.000000,0.000000,0.000000,0.220258,0.070700,0.101971
353,0.552361,0.000000,0.100616,0.000000,0.000000,0.000000,0.254620,0.092402,0.000000
354,0.340197,0.000000,0.179946,0.000000,0.000000,0.000000,0.479857,0.000000,0.000000


In [260]:
# n_municipalities = data['size'][data['id'].index('Region')]
n_party = data['size'][data['id'].index('PartiValglistebm')]
votes = iter(data['value'])
party_names = list(dims['PartiValglistebm']['category']['label'].values())

def norm_party(df):
    df_ret = df.copy()
    df_ret[party_names] = df[party_names].div(np.maximum(df[party_names].sum(axis=1), 1), axis=0)
    return df_ret

df = (
    pd.DataFrame(
        (party_vote for party_vote in grouper(n_party, votes)),
        columns=party_names,
    )

    .assign(
        stemmer_totalt=lambda x: x.sum(axis=1),
        kommunenummer=dims['Region']['category']['label'].keys(),
        kommunenavn=dims['Region']['category']['label'].values(),
    )
    .transform(norm_party)
    .reorder_columns(['kommunenummer', 'kommunenavn'])    
)

# df.sort_values('stemmer_totalt')
# df[party_names]

In [262]:
df.head()

,kommunenummer,kommunenavn,Arbeiderpartiet,Fremskrittspartiet,Høyre,Kristelig Folkeparti,Miljøpartiet de Grønne,Rødt,Senterpartiet,Sosialistisk Venstreparti,Venstre,stemmer_totalt
0,3001,Halden,0.192359,0.076411,0.306015,0.035324,0.078185,0.077150,0.150532,0.054981,0.029042,13532
1,3002,Moss,0.312092,0.098618,0.214563,0.027223,0.066376,0.079585,0.081432,0.067749,0.052362,21122
2,3003,Sarpsborg,0.424196,0.127499,0.120590,0.049196,0.048804,0.051842,0.132840,0.029743,0.015288,20408
3,3004,Fredrikstad,0.394290,0.136044,0.165726,0.038340,0.075261,0.054209,0.072134,0.048099,0.015898,34533
4,3005,Drammen,0.289306,0.118372,0.283716,0.024843,0.095036,0.027813,0.088123,0.049687,0.023104,43110


In [263]:
df.to_parquet('data/valg2019.parquet')